### 1. data processing

In [1]:
import pandas as pd
import numpy as np
from eunjeon import Mecab
import json

In [2]:
mecab = Mecab()

In [3]:
def txt_to_csv(filepath):
    df = pd.DataFrame(columns=["sentence", "emotion"])
    with open(filepath, 'r', encoding="UTF8") as f:
        s = json.load(f)
        for i, data in enumerate(s):
            df.loc[i] = [data["review"], data["rating"]]
    return df

In [4]:
data = pd.DataFrame(columns=["sentence", "emotion"])

for i in range(10001, 142620):
    print(i, end="\r")
    try:
        data = data.append(txt_to_csv(f"./datasets/raw/{i}.json"))
    except:
        continue
data

,sentence,emotion
0,전체관람가는 아닌것 같아요,10
1,디렉터스컷으로봐서 거의 3시간짜리인데 참 흡인력있다,10
2,태어나 처음으로 가슴아리는 영화였다. 20년이상 지났지만.. 생각하면 또 가슴이...,10
3,어린시절 고딩때 봤던 때랑 또 결혼하고 나서 봤을때의 느낌은 확실히 다르네요. 뭔가...,10
4,토토에게 넓은 세상을 보여주고픈 알프레도.. 그가 토토를 위해 정을 떼려고 했던 장...,10
...,...,...
0,재미잇내요. 주인공이 너무무능력하내,10
0,재미있게생겼다.엄청,10
0,해맑은 젖통이ㅇ얌ㅇㅇ,10
0,"이분들은 우리나라를 대표하는 크리에이터,,,...",10


In [5]:
data.reset_index(drop=True, inplace=True)
data.head()

,sentence,emotion
0,전체관람가는 아닌것 같아요,10
1,디렉터스컷으로봐서 거의 3시간짜리인데 참 흡인력있다,10
2,태어나 처음으로 가슴아리는 영화였다. 20년이상 지났지만.. 생각하면 또 가슴이...,10
3,어린시절 고딩때 봤던 때랑 또 결혼하고 나서 봤을때의 느낌은 확실히 다르네요. 뭔가...,10
4,토토에게 넓은 세상을 보여주고픈 알프레도.. 그가 토토를 위해 정을 떼려고 했던 장...,10


In [6]:
data["sentence"] = data['sentence'].apply(lambda x: np.str_(x))

def apply_emotion(x):
    if int(x) > 7: return 1
    else: return 0

data["emotion"] = data["emotion"].apply(lambda x: apply_emotion(x))

In [7]:
data.isna().sum()

sentence    0
emotion     0
dtype: int64

In [8]:
data.to_csv("./datasets/sentences.csv")

### 2. Main(fit, predict)

In [9]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("./datasets/sentences.csv", index_col=0)
data.head()

,sentence,emotion
0,전체관람가는 아닌것 같아요,1
1,디렉터스컷으로봐서 거의 3시간짜리인데 참 흡인력있다,1
2,태어나 처음으로 가슴아리는 영화였다. 20년이상 지났지만.. 생각하면 또 가슴이...,1
3,어린시절 고딩때 봤던 때랑 또 결혼하고 나서 봤을때의 느낌은 확실히 다르네요. 뭔가...,1
4,토토에게 넓은 세상을 보여주고픈 알프레도.. 그가 토토를 위해 정을 떼려고 했던 장...,1


In [10]:
data["sentence"] = data["sentence"].fillna("0")

In [11]:
features = data["sentence"]
label = data["emotion"]
train_x, test_x, train_y, test_y = train_test_split(features, label, test_size=0.2, random_state=0)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(tokenizer = mecab.morphs, ngram_range=(1, 2), min_df=3, max_df=0.9)
# x = tfv.fit_transform(train_x.as_type("U"))
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<569921x325488 sparse matrix of type '<class 'numpy.float64'>'
	with 17449591 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0, solver='liblinear')
params = {"C": [1, 3, 5, 7, 9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv = 3, scoring="accuracy", verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  6.5min finished


GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=0, solver='liblinear'),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [20]:
tfv_test_x = tfv.transform(test_x)
pred = grid_cv.predict(tfv_test_x)
pred[:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 0, 0], dtype=int64)

In [21]:
from sklearn.metrics import accuracy_score
import numpy as np

accuracy = np.round(accuracy_score(test_y, pred), 4)
print("accuracy score:", accuracy)

accuracy score: 0.8236


In [22]:
import joblib

joblib.dump(grid_cv.best_estimator_, "./analysis_model.pkl")

['./analysis_model.pkl']